# Obtain data of AGN from SIMBAD and filtering Ne II and Ne III Passband from NED data

The code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

Here we use [Astroquery](https://astroquery.readthedocs.io/en/latest/index.html) package to retriev the data.

In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 802 kB 60.4 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 3.5 MB 44.5 MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-py3-none-any.whl size=3864 sha256=5c9f325a9fa82864ce2702cea75499139dbe0a6d908dc3d7534285a7d5e904e5
  Stored in directory: /root/.cache/pip/wheels/49/b4/2d/0081759ae1833bd694024801f7aacddcda8a687e8d5fbaeebd
Successfully built mimeparse


In [2]:
from datetime import date

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [3]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [4]:
today = date.today()

The sample results is saved in a .csv file:

In [5]:
simbad_sample_Sy = result.to_table()
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)
len(simbad_sample_Sy)

43708

In [35]:
simbad_sample_Sy[33]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
SDSS J085416.75+502631.9,133.56973333333335,50.44226944444444,2006AJ....131.1163S,Sy2,2016ApJ...832...67N,27314.34577581347,0.095668,z,14,Sy2|AGN|G


Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because when we search the information in [NED](https://ned.ipac.caltech.edu/) using the object id that we obtained from SIMBAD, sometimes it doesn't reconigze it, so we will use coordinates instead

In [6]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

In the next part of the code we are going to filter the samples that have Ne II, Ne III and Ne V data. First we define a variable with the names of the **"Observed Passband"** for Ne II, Ne III and Ne V. Then with the first loop we will get the photometry table for each object sample, specifically the "Observed Passband Colum" of that tables for search in the second loop all the objects that have data in the Ne passbands

In [150]:
Ne_PB=['[Ne II] 12.81 (IRS)','[Ne II] 12.8 Spitzer','[Ne II] 12.8 (IRS)','[Ne V] 14.32 (IRS)',
       '[Ne V] 14.32 Spitzer','[Ne V] 14.32','[Ne V] 14.32 (IRS)','[Ne V] 14.3 Spitzer',
       '[Ne III] 15.56 (IRS)','[NeIII] 15.6 Spitzer','[Ne III] 15.56 (IRS)','[Ne III] (Spitzer)',
       '[Ne V] 14.3 (IRS)','[Ne V] 14.3 microns','[Ne III] 15.5 (IRS)','[NeIII] 15.6 microns']

Ne_IR_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

total_obj_Ne=0

for i in range(2000):
    
    result_table = Ned.query_region(cat_smb_sample[i],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample
    #print(result_table) # an astropy.table.Table
    obj_name=result_table["Object Name"] #Preferred object name in NED
    k=i
    try:
      phot_table = Ned.get_table(obj_name, table='photometry') #We get the photometry table from NED for each sample
      len_table = len(phot_table) #Lenght of the photometry data

      for j in range(len_table):
        Freq=phot_table["Frequency"][j] #We get the name for each Observed Passband
        
        if (Freq==Ne_IR_Freq[0] or Freq==Ne_IR_Freq[1] or Freq==Ne_IR_Freq[2] or Freq==Ne_IR_Freq[3] or 
            Freq==Ne_IR_Freq[4]): 
          
          if k==i:
            total_obj_Ne+=1
            print(i)
            print(obj_name)
            print(phot_table["Observed Passband","Photometry Measurement","Frequency"][j]) 
          else:
            print(phot_table["Observed Passband","Photometry Measurement","Frequency"][j])
            
          k+=1  
          print('\n')
    
    except:
      pass 

print('Total objects with Ne II or Ne III or Ne V data found: ',total_obj_Ne)   

0
       Object Name       
-------------------------
WISEA J140931.24+051131.6
 Observed Passband  Photometry Measurement    Frequency    
                                                  Hz       
------------------- ---------------------- ----------------
[Ne II] 12.81 (IRS)               1.33e-21 23400000000000.0


Observed Passband  Photometry Measurement    Frequency    
                                                 Hz       
------------------ ---------------------- ----------------
[Ne V] 14.32 (IRS)                     -- 20900000000000.0


 Observed Passband  Photometry Measurement    Frequency    
                                                  Hz       
------------------- ---------------------- ----------------
[Cl II] 14.37 (IRS)                     -- 20900000000000.0


 Observed Passband   Photometry Measurement    Frequency    
                                                   Hz       
-------------------- ---------------------- ----------------
[Ne III] 15.56 

In [140]:
table=simbad_sample_Sy[0]
table

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
LEDA 165630,212.38023750000002,5.192030555555555,2009ApJS..182..543A,Sy2,2011A&A...528A.124C,69068.1659042559,0.2644,z,68,IR |QSO|Sy2|G |G


In [141]:
ned_result = Ned.query_object(table[0])

In [149]:
result_t = Ned.query_region(cat_smb_sample[101],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 

o_name=result_t["Object Name"]
try:
  phot_t = Ned.get_table(o_name, table='photometry')
  #print(phot_t['Frequency'])
  len_t=len(phot_t)
  for a in range(len_t):
    Passb=phot_t['Frequency'][a] #We get the name for each Observed Passband
    if (Passb==Ne_IR_Freq[0] or Passb==Ne_IR_Freq[1] or Passb==Ne_IR_Freq[2] or Passb==Ne_IR_Freq[3] or Passb==Ne_IR_Freq[4]):
      print(o_name)
except:
  pass

Object Name
-----------
     3C 334
Object Name
-----------
     3C 334
Object Name
-----------
     3C 334
Object Name
-----------
     3C 334
Object Name
-----------
     3C 334
